## 1. Import Libraries và Load Data

In [2]:
# Import thư viện cần thiết
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
warnings.filterwarnings('ignore')

# TF-IDF và cosine similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

# Word Embedding
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
# Load dữ liệu
movies_metadata = pd.read_csv('movies_lab3/movies_metadata.csv', low_memory=False)
ratings = pd.read_csv('movies_lab3/ratings.csv')
links = pd.read_csv('movies_lab3/links.csv')

print(f"Movies metadata shape: {movies_metadata.shape}")
print(f"Ratings shape: {ratings.shape}")
print(f"Links shape: {links.shape}")
print("Movies metadata sample:")
display(movies_metadata)

Movies metadata shape: (45466, 24)
Ratings shape: (26024289, 4)
Links shape: (45843, 3)
Movies metadata sample:


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


## 2. Lấy tất cả phim của UserID = 201 có overview

In [4]:
# Lọc ratings của user 201
user_201_ratings = ratings[ratings['userId'] == 201].copy()
print(f"Số lượng phim user 201 đã đánh giá: {len(user_201_ratings)}")
print("\nDữ liệu ratings của user 201:")
display(user_201_ratings.head(10))

Số lượng phim user 201 đã đánh giá: 370

Dữ liệu ratings của user 201:


,userId,movieId,rating,timestamp
15964,201,1,4.5,1166795290
15965,201,11,3.0,1166795833
15966,201,16,4.0,1075438622
15967,201,19,3.5,1075438555
15968,201,21,3.5,1077456157
15969,201,25,4.0,1086725975
15970,201,32,3.5,1086725786
15971,201,36,4.5,1077456160
15972,201,39,5.0,1103910174
15973,201,47,5.0,1086725334


### Code merge trực tiếp không thông qua links.csv (QUAN TRỌNG)

In [5]:
'''
# Chuẩn hoá ID trong movies_metadata
movies_metadata['id'] = pd.to_numeric(movies_metadata['id'], errors='coerce')
movies_metadata = movies_metadata.dropna(subset=['id'])
movies_metadata['id'] = movies_metadata['id'].astype(int)

print(f"Movies metadata cleaned: {len(movies_metadata)} phim có ID hợp lệ")

# Chuẩn hoá movieId trong ratings
user_201_ratings['movieId'] = pd.to_numeric(user_201_ratings['movieId'], errors='coerce')
user_201_ratings = user_201_ratings.dropna(subset=['movieId'])
user_201_ratings['movieId'] = user_201_ratings['movieId'].astype(int)

print(f"Ratings cleaned: {len(user_201_ratings)} dòng rating hợp lệ")

# Merge trực tiếp movieId <-> id
direct_merged = user_201_ratings.merge(
    movies_metadata[['id', 'title', 'overview']],
    left_on='movieId',
    right_on='id',
    how='inner'
)

print(f"Số phim match trực tiếp giữa ratings và metadata: {len(direct_merged)}")

# Lọc overview hợp lệ
direct_overview = direct_merged[
    direct_merged['overview'].notna() &
    (direct_merged['overview'].str.strip() != '')
].copy()

print(f"Số phim có overview hợp lệ: {len(direct_overview)}")

print("\nSample phim có overview hợp lệ:")
display(direct_overview[['movieId', 'title', 'rating', 'overview']].head())
'''


'\n# Chuẩn hoá ID trong movies_metadata\nmovies_metadata[\'id\'] = pd.to_numeric(movies_metadata[\'id\'], errors=\'coerce\')\nmovies_metadata = movies_metadata.dropna(subset=[\'id\'])\nmovies_metadata[\'id\'] = movies_metadata[\'id\'].astype(int)\n\nprint(f"Movies metadata cleaned: {len(movies_metadata)} phim có ID hợp lệ")\n\n# Chuẩn hoá movieId trong ratings\nuser_201_ratings[\'movieId\'] = pd.to_numeric(user_201_ratings[\'movieId\'], errors=\'coerce\')\nuser_201_ratings = user_201_ratings.dropna(subset=[\'movieId\'])\nuser_201_ratings[\'movieId\'] = user_201_ratings[\'movieId\'].astype(int)\n\nprint(f"Ratings cleaned: {len(user_201_ratings)} dòng rating hợp lệ")\n\n# Merge trực tiếp movieId <-> id\ndirect_merged = user_201_ratings.merge(\n    movies_metadata[[\'id\', \'title\', \'overview\']],\n    left_on=\'movieId\',\n    right_on=\'id\',\n    how=\'inner\'\n)\n\nprint(f"Số phim match trực tiếp giữa ratings và metadata: {len(direct_merged)}")\n\n# Lọc overview hợp lệ\ndirect_overv

##### Cell code phía trên là khi em merge trực tiếp movieID từ ratings với id từ movies_metadata mà không dùng đến links.csv. Em nghĩ mọi người ra ít phim có overview hơn số 370 (như ở trên em ra 165) là do mọi người đang merge bảng bị SAI. Em xin phân tích thêm:
1. movieId trong ratings.csv = MovieLens internal ID (1, 2, 3, 4, 5, ...)
2. id trong movies_metadata.csv = TMDB ID (862, 8844, 15602, ...)
##### -> Hai hệ thống này dù nhìn thì có vẻ giống nhưng chúng HOÀN TOÀN KHÁC NHAU. Để chứng minh kỹ hơn thì có thể dùng dòng lệnh movies_metadata[movies_metadata['id'] == 1] và chạy thì sẽ không trích được bản ghi nào, tuy nhiên rõ ràng trong user_201_ratings phía trên in ra có id 1.

##### Nếu đúng thì pipeline sẽ là:
ratings.csv : userID: 201; movieID: 1; rating: 4.0 ->
links.csv: movieID ;tmdbID 862 ->
movies_metadata.csv: id (tmdbID); title; overview; genres...

### MERGE đúng

In [6]:
# Clean dữ liệu trước khi merge
# 1. Clean movies_metadata['id'] - chuyển sang int
movies_metadata['id'] = pd.to_numeric(movies_metadata['id'], errors='coerce')
movies_metadata = movies_metadata.dropna(subset=['id'])
movies_metadata['id'] = movies_metadata['id'].astype(int)

# 2. Clean links['tmdbId'] - chuyển sang int
links['tmdbId'] = pd.to_numeric(links['tmdbId'], errors='coerce')
links = links.dropna(subset=['tmdbId'])
links['tmdbId'] = links['tmdbId'].astype(int)

print(f"Movies metadata cleaned: {len(movies_metadata)} phim có ID hợp lệ")
print(f"Links cleaned: {len(links)} phim có TMDB ID hợp lệ")

# Sắp xếp theo timestamp
user_201_ratings = user_201_ratings.sort_values('timestamp').reset_index(drop=True)

# Merge với links để có tmdbId
user_201_with_tmdb = user_201_ratings.merge(links[['movieId', 'tmdbId']], on='movieId', how='left')

print(f"Số phim có TMDB ID: {user_201_with_tmdb['tmdbId'].notna().sum()}/{len(user_201_with_tmdb)}")

# Merge với movies_metadata để lọc những phim có overview
user_201_with_overview = user_201_with_tmdb.merge(
    movies_metadata[['id', 'title', 'overview']], 
    left_on='tmdbId', 
    right_on='id', 
    how='inner'  # Inner join để chỉ lấy phim có metadata
)

# Lọc những phim có overview (không null và không rỗng)
user_201_with_overview = user_201_with_overview[
    user_201_with_overview['overview'].notna() & 
    (user_201_with_overview['overview'].str.strip() != '')
].copy()

print(f"Số phim có overview hợp lệ: {len(user_201_with_overview)}")
print("\nSample phim có overview:")
display(user_201_with_overview[['movieId', 'title', 'rating', 'overview']].head())

Movies metadata cleaned: 45463 phim có ID hợp lệ
Links cleaned: 45624 phim có TMDB ID hợp lệ
Số phim có TMDB ID: 370/370
Số phim có overview hợp lệ: 370

Sample phim có overview:


,movieId,title,rating,overview
0,19,Ace Ventura: When Nature Calls,3.5,"Summoned from an ashram in Tibet, Ace finds hi..."
1,1961,Rain Man,4.5,Selfish yuppie Charlie Babbitt's father left a...
2,1220,The Blues Brothers,5.0,"Jake Blues is just out of jail, and teams up w..."
3,2355,A Bug's Life,4.0,"On behalf of ""oppressed bugs everywhere,"" an i..."
4,104,Happy Gilmore,4.0,Failed hockey player-turned-golf whiz Happy Gi...


## 3. Chia tập Test (90%) và Profile (10%)

In [7]:
# Chia theo tỉ lệ 90% : 10%
total_movies = len(user_201_with_overview)
test_size = int(total_movies * 0.9)  # 90% cho test
profile_size = total_movies - test_size  # 10% cho profile

# Cắt 90% đầu làm test set
test_set = user_201_with_overview.head(test_size).copy()

# 10% còn lại làm profile set
profile_set = user_201_with_overview.tail(profile_size).copy()

print(f"Tổng số phim có overview: {total_movies}")
print(f"Test set size (90%): {len(test_set)}")
print(f"Profile set size (10%): {len(profile_set)}")

print("Profile Set (10% cuối để biểu diễn user 201)")
display(profile_set[['movieId', 'title', 'rating', 'overview']])

print("Test Set (90% đầu) - Sample")
display(test_set[['movieId', 'title', 'rating', 'overview']].head(10))

Tổng số phim có overview: 370
Test set size (90%): 333
Profile set size (10%): 37
Profile Set (10% cuối để biểu diễn user 201)


,movieId,title,rating,overview
333,852,Tin Cup,4.0,A washed up golf pro working at a driving rang...
334,3527,Predator,2.5,Dutch and his group of commandos are hired by ...
335,1302,Field of Dreams,3.5,Ray Kinsella is an Iowa farmer who hears a mys...
336,2470,Crocodile Dundee,1.5,When a New York reporter plucks crocodile hunt...
337,105,The Bridges of Madison County,2.5,Photographer Robert Kincaid wanders into the l...
338,1092,Basic Instinct,4.0,A police detective is in charge of the investi...
339,785,Kingpin,4.0,After bowler Roy Munson swindles the wrong cro...
340,333,Tommy Boy,5.0,Party animal Tommy Callahan is a few cans shor...
341,3448,"Good Morning, Vietnam",3.5,Radio funny man Adrian Cronauer is sent to Vie...
342,1747,Wag the Dog,2.5,"During the final weeks of a presidential race,..."


Test Set (90% đầu) - Sample


,movieId,title,rating,overview
0,19,Ace Ventura: When Nature Calls,3.5,"Summoned from an ashram in Tibet, Ace finds hi..."
1,1961,Rain Man,4.5,Selfish yuppie Charlie Babbitt's father left a...
2,1220,The Blues Brothers,5.0,"Jake Blues is just out of jail, and teams up w..."
3,2355,A Bug's Life,4.0,"On behalf of ""oppressed bugs everywhere,"" an i..."
4,104,Happy Gilmore,4.0,Failed hockey player-turned-golf whiz Happy Gi...
5,1923,There's Something About Mary,5.0,Having never fully recovered from a prom date ...
6,2710,The Blair Witch Project,3.5,In October of 1994 three student filmmakers di...
7,16,Casino,4.0,The life of the gambling paradise – Las Vegas ...
8,1517,Austin Powers: International Man of Mystery,5.0,As a swingin' fashion photographer by day and ...
9,1080,Life of Brian,4.0,"Brian Cohen is an average young Jewish man, bu..."


## 4. Tiền xử lý phần Overview

In [8]:
# Hàm tiền xử lý text
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    if not isinstance(text, str) or pd.isna(text):
        return ''
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Áp dụng preprocessing cho movies_metadata
movies_metadata['overview_clean'] = movies_metadata['overview'].apply(preprocess_text)
movies_metadata['overview_clean'] = movies_metadata['overview_clean'].fillna('')

print("Sample preprocessed overviews:")
for idx in range(3):
    print(f"\nOriginal: {movies_metadata['overview'].iloc[idx][:100]}...")
    print(f"Cleaned: {movies_metadata['overview_clean'].iloc[idx][:100]}...")

Sample preprocessed overviews:

Original: Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto ...
Cleaned: led by woody andys toys live happily in his room until andys birthday brings buzz lightyear onto the...

Original: When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world...
Cleaned: when siblings judy and peter discover an enchanted board game that opens the door to a magical world...

Original: A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and...
Cleaned: a family wedding reignites the ancient feud between nextdoor neighbors and fishing buddies john and ...


## 5. Tạo User Profile từ Profile Set

In [9]:
# Chuẩn bị dữ liệu cho evaluation
# profile_set và test_set đã có đầy đủ thông tin từ bước trước
profile_movies = profile_set[['userId', 'movieId', 'rating', 'timestamp', 'tmdbId']].copy()
test_movies = test_set[['userId', 'movieId', 'rating', 'timestamp', 'tmdbId']].copy()

# profile_movies_metadata chứa đầy đủ thông tin metadata
profile_movies_metadata = profile_set[['userId', 'movieId', 'rating', 'timestamp', 'tmdbId', 'id', 'title', 'overview']].copy()

print(f"Profile set: {len(profile_movies)} phim")
print(f"Test set: {len(test_movies)} phim")
print(f"Profile có metadata: {len(profile_movies_metadata)} phim")

# Kết hợp overview của tất cả phim trong profile set để tạo user profile
profile_overviews = profile_movies_metadata['overview'].values

# Tiền xử lý các overview này
profile_overviews_clean = [preprocess_text(overview) for overview in profile_overviews]
profile_overviews_clean = [ov for ov in profile_overviews_clean if ov]  # Remove empty

# Kết hợp tất cả thành một văn bản đại diện cho user 201
user_profile_text = ' '.join(profile_overviews_clean)

print(f"User Profile")
print(f"Số phim có overview trong profile: {len(profile_overviews_clean)}")
print(f"User profile text length: {len(user_profile_text)} characters")

Profile set: 37 phim
Test set: 333 phim
Profile có metadata: 37 phim
User Profile
Số phim có overview trong profile: 37
User profile text length: 11494 characters


## 6. PHƯƠNG PHÁP 1: TF-IDF Vectorization

### 6.1. Vector hóa bằng TF-IDF

In [10]:
# Tạo TF-IDF vectorizer
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)

# Fit trên toàn bộ movies corpus
tfidf_matrix = tfidf.fit_transform(movies_metadata['overview_clean'])

print(f"TF-IDF matrix shape: {tfidf_matrix.shape}")
print(f"Number of features: {len(tfidf.get_feature_names_out())}")

TF-IDF matrix shape: (45463, 5000)
Number of features: 5000


In [11]:
# Transform user profile
user_profile_tfidf = tfidf.transform([user_profile_text])

print(f"User profile TF-IDF shape: {user_profile_tfidf.shape}")
print(f"User profile TF-IDF non-zero elements: {user_profile_tfidf.nnz}")

User profile TF-IDF shape: (1, 5000)
User profile TF-IDF non-zero elements: 592


### 6.2. Tính Cosine Similarity và Recommend Top 100

In [12]:
# Tính cosine similarity giữa user profile và tất cả movies
cosine_similarities_tfidf = cosine_similarity(user_profile_tfidf, tfidf_matrix).flatten()

# Tạo DataFrame với similarity scores
movies_with_scores_tfidf = movies_metadata.copy()
movies_with_scores_tfidf['similarity_score'] = cosine_similarities_tfidf

# Sắp xếp theo similarity score giảm dần
movies_with_scores_tfidf = movies_with_scores_tfidf.sort_values('similarity_score', ascending=False)

print("Top 10 movies theo TF-IDF similarity:")
display(movies_with_scores_tfidf[['id', 'title', 'similarity_score']].head(10))

Top 10 movies theo TF-IDF similarity:


,id,title,similarity_score
30315,150540,Inside Out,0.341339
3456,98,Gladiator,0.307463
13187,13223,Gran Torino,0.255390
6638,891,All the President's Men,0.254032
3406,106,Predator,0.219050
11437,1372,Blood Diamond,0.212795
226,11929,Dolores Claiborne,0.212670
1683,11003,The Wedding Singer,0.211597
12704,10681,WALL·E,0.203866
2294,10328,Cocoon,0.202644


In [13]:
# Lấy top 100 movies được recommend (dự đoán)
# Loại bỏ các phim đã có trong profile set
profile_movie_ids = set(profile_movies_metadata['id'].dropna().astype(int))

# Filter ra các phim không thuộc profile
movies_not_in_profile_tfidf = movies_with_scores_tfidf[
    ~movies_with_scores_tfidf['id'].isin(profile_movie_ids)
]

# Lấy top 100
top_100_recommendations_tfidf = movies_not_in_profile_tfidf.head(100)

print("\nTop 10 recommendations:")
display(top_100_recommendations_tfidf[['id', 'title', 'similarity_score']].head(10))


Top 10 recommendations:


,id,title,similarity_score
36124,116306,Exit Elena,0.176683
35379,363480,Christmas Incorporated,0.165140
13688,24053,The Merry Gentleman,0.161737
33900,125232,The Ball at the Anjo House,0.159864
39572,216043,The Iceman: Confessions of a Mafia Hitman,0.158272
20224,66193,Sinners and Saints,0.153357
29321,283698,Hungry Hearts,0.151780
15388,70804,J'accuse!,0.149919
20937,55576,Last Stand at Saber River,0.149470
31927,97938,Heart of the Country,0.148560


## 7. PHƯƠNG PHÁP 2: Word Embedding (Word2Vec)

### 7.1. Tokenize và Train Word2Vec

In [14]:
# Tokenize tất cả overviews
def tokenize_text(text):
    if not isinstance(text, str) or not text:
        return []
    return word_tokenize(text.lower())

# Tokenize toàn bộ corpus
tokenized_corpus = movies_metadata['overview_clean'].apply(tokenize_text).tolist()

# Remove empty lists
tokenized_corpus = [tokens for tokens in tokenized_corpus if tokens]

print(f"Number of documents: {len(tokenized_corpus)}")
print(f"Sample tokens: {tokenized_corpus[0][:20]}")

Number of documents: 44491
Sample tokens: ['led', 'by', 'woody', 'andys', 'toys', 'live', 'happily', 'in', 'his', 'room', 'until', 'andys', 'birthday', 'brings', 'buzz', 'lightyear', 'onto', 'the', 'scene', 'afraid']


In [15]:
# Train Word2Vec model
w2v_model = Word2Vec(
    sentences=tokenized_corpus,
    vector_size=100,  # Dimension of word vectors
    window=5,         # Context window size
    min_count=2,      # Ignore words with frequency < 2
    workers=4,        # Number of threads
    sg=1,             # Skip-gram model
    epochs=10
)

print(f"Vocabulary size: {len(w2v_model.wv)}")
print(f"Vector dimension: {w2v_model.wv.vector_size}")

Vocabulary size: 45422
Vector dimension: 100


### 7.2. Tạo Document Vectors

In [16]:
def get_document_vector(tokens, model):
    vectors = []
    for token in tokens:
        if token in model.wv:
            vectors.append(model.wv[token])
    
    if not vectors:
        return np.zeros(model.wv.vector_size)
    
    return np.mean(vectors, axis=0)

# Tạo vectors cho tất cả movies
movies_metadata['tokens'] = movies_metadata['overview_clean'].apply(tokenize_text)
movies_metadata['doc_vector'] = movies_metadata['tokens'].apply(
    lambda tokens: get_document_vector(tokens, w2v_model)
)

print("Sample vector shape:", movies_metadata['doc_vector'].iloc[0].shape)

Sample vector shape: (100,)


In [17]:
# Tạo user profile vector từ 10 phim
user_profile_tokens = tokenize_text(user_profile_text)
user_profile_w2v = get_document_vector(user_profile_tokens, w2v_model)

print(f"User profile W2V shape: {user_profile_w2v.shape}")
print(f"User profile W2V sample values: {user_profile_w2v[:10]}")

User profile W2V shape: (100,)
User profile W2V sample values: [-0.06766643  0.17009282  0.04015268  0.06258554  0.05738945 -0.24092592
  0.24486083  0.43312854 -0.17230959 -0.22561707]


### 7.3. Tính Cosine Similarity và Recommend Top 100

In [18]:
# Tạo matrix từ doc vectors
doc_vectors_matrix = np.vstack(movies_metadata['doc_vector'].values)

print(f"Document vectors matrix shape: {doc_vectors_matrix.shape}")

# Tính cosine similarity
user_profile_w2v_reshaped = user_profile_w2v.reshape(1, -1)
cosine_similarities_w2v = cosine_similarity(user_profile_w2v_reshaped, doc_vectors_matrix).flatten()

print(f"Cosine similarities shape: {cosine_similarities_w2v.shape}")
print(f"Sample similarities: {cosine_similarities_w2v[:10]}")

Document vectors matrix shape: (45463, 100)
Cosine similarities shape: (45463,)
Sample similarities: [0.948576   0.9527597  0.95388039 0.96670973 0.95460343 0.95919973
 0.91338162 0.94999821 0.93635492 0.9294001 ]


In [19]:
# Tạo DataFrame với W2V similarity scores
movies_with_scores_w2v = movies_metadata.copy()
movies_with_scores_w2v['similarity_score'] = cosine_similarities_w2v

# Sắp xếp theo similarity
movies_with_scores_w2v = movies_with_scores_w2v.sort_values('similarity_score', ascending=False)

print("Top 10 movies theo Word2Vec similarity:")
display(movies_with_scores_w2v[['id', 'title', 'similarity_score']].head(10))

Top 10 movies theo Word2Vec similarity:


,id,title,similarity_score
23414,260202,Turks & Caicos,0.988427
34798,268413,Antidote,0.987935
19272,24410,K-20: The Fiend with Twenty Faces,0.987409
41274,393603,The Congressman,0.987108
855,12559,First Kid,0.987064
15005,27936,Micmacs,0.987051
355,10879,I Love Trouble,0.987010
9853,37748,800 Bullets,0.986896
19777,20528,The Ambush: Incident at Blood Pass,0.986672
41671,98596,Overdose,0.986502


In [20]:
# Lấy top 100 recommendations (loại bỏ profile movies)
movies_not_in_profile_w2v = movies_with_scores_w2v[
    ~movies_with_scores_w2v['id'].isin(profile_movie_ids)
]

top_100_recommendations_w2v = movies_not_in_profile_w2v.head(100)

print("\nTop 10 recommendations:")
display(top_100_recommendations_w2v[['id', 'title', 'similarity_score']].head(10))


Top 10 recommendations:


,id,title,similarity_score
23414,260202,Turks & Caicos,0.988427
34798,268413,Antidote,0.987935
19272,24410,K-20: The Fiend with Twenty Faces,0.987409
41274,393603,The Congressman,0.987108
855,12559,First Kid,0.987064
15005,27936,Micmacs,0.987051
355,10879,I Love Trouble,0.987010
9853,37748,800 Bullets,0.986896
19777,20528,The Ambush: Incident at Blood Pass,0.986672
41671,98596,Overdose,0.986502


## 8. Đánh giá: P@K, R@K, F1@K (K=100)

### 8.1. Chuẩn bị Test Set với TMDB IDs

In [21]:
# Lấy TMDB IDs từ test set
test_movies_ids = set(test_movies['tmdbId'].dropna().astype(int))

print(f"Số phim trong test set có TMDB ID: {len(test_movies_ids)}")
print(f"Sample test movie IDs: {list(test_movies_ids)[:10]}")

Số phim trong test set có TMDB ID: 333
Sample test movie IDs: [2054, 4107, 12, 13, 14, 15, 24, 28, 33, 2105]


### 8.2. Tính Precision@K, Recall@K, F1@K

In [22]:
def calculate_metrics_at_k(recommended_ids, test_ids, k=100):
    # Số phim được recommend đúng (có trong test set)
    true_positives = len(recommended_ids.intersection(test_ids))
    
    # Precision@K = TP / K
    precision = true_positives / k if k > 0 else 0
    
    # Recall@K = TP / |test set|
    recall = true_positives / len(test_ids) if len(test_ids) > 0 else 0
    
    # F1@K
    if precision + recall > 0:
        f1 = 2 * (precision * recall) / (precision + recall)
    else:
        f1 = 0
    
    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'true_positives': true_positives
    }

In [23]:
# Tính metrics cho TF-IDF
recommended_ids_tfidf = set(top_100_recommendations_tfidf['id'].astype(int))

metrics_tfidf = calculate_metrics_at_k(recommended_ids_tfidf, test_movies_ids, k=100)

print("METRICS CHO TF-IDF (K=100)")
print(f"True Positives: {metrics_tfidf['true_positives']}")
print(f"Precision@100: {metrics_tfidf['precision']:.4f}")
print(f"Recall@100: {metrics_tfidf['recall']:.4f}")
print(f"F1@100: {metrics_tfidf['f1']:.4f}")

METRICS CHO TF-IDF (K=100)
True Positives: 0
Precision@100: 0.0000
Recall@100: 0.0000
F1@100: 0.0000


In [24]:
# Tính metrics cho Word2Vec
recommended_ids_w2v = set(top_100_recommendations_w2v['id'].astype(int))

metrics_w2v = calculate_metrics_at_k(recommended_ids_w2v, test_movies_ids, k=100)

print("METRICS CHO WORD2VEC (K=100)")
print(f"True Positives: {metrics_w2v['true_positives']}")
print(f"Precision@100: {metrics_w2v['precision']:.4f}")
print(f"Recall@100: {metrics_w2v['recall']:.4f}")
print(f"F1@100: {metrics_w2v['f1']:.4f}")

METRICS CHO WORD2VEC (K=100)
True Positives: 1
Precision@100: 0.0100
Recall@100: 0.0030
F1@100: 0.0046


## 9. Đánh giá: MRR và NDCG

### 9.1. Sắp xếp Test Set theo Rating

In [25]:
# Sắp xếp test set theo rating giảm dần
test_sorted = test_movies.sort_values('rating', ascending=False).reset_index(drop=True)

print("Test set đã sắp xếp theo rating:")
display(test_sorted[['movieId', 'tmdbId', 'rating', 'timestamp']].head(20))

# Tạo list các tmdbId theo thứ tự rating giảm dần
test_sorted_ids = test_sorted['tmdbId'].dropna().astype(int).tolist()
print(f"\nSố phim trong test set có TMDB ID: {len(test_sorted_ids)}")

Test set đã sắp xếp theo rating:


,movieId,tmdbId,rating,timestamp
0,953,1585,5.0,1166796321
1,6796,650,5.0,1075440028
2,2762,745,5.0,1077455963
3,3948,1597,5.0,1075869694
4,1079,623,5.0,1075869712
5,6377,12,5.0,1166794799
6,7018,11092,5.0,1075439694
7,1228,1578,5.0,1075869641
8,1242,9665,5.0,1166794593
9,1090,792,5.0,1166794661



Số phim trong test set có TMDB ID: 333


### 9.2. Tính MRR (Mean Reciprocal Rank)

In [26]:
def calculate_mrr(recommended_list, relevant_list):
    relevant_set = set(relevant_list)
    
    for rank, movie_id in enumerate(recommended_list, start=1):
        if movie_id in relevant_set:
            return 1.0 / rank
    
    return 0.0

# Test
test_mrr = calculate_mrr([1, 2, 3, 4, 5], [3])
print(f"Test MRR (relevant at position 3): {test_mrr}")

Test MRR (relevant at position 3): 0.3333333333333333


In [27]:
# Tính MRR cho TF-IDF
recommended_list_tfidf = top_100_recommendations_tfidf['id'].astype(int).tolist()
mrr_tfidf = calculate_mrr(recommended_list_tfidf, test_sorted_ids)

print(f"MRR for TF-IDF: {mrr_tfidf:.4f}")

MRR for TF-IDF: 0.0000


In [28]:
# Tính MRR cho Word2Vec
recommended_list_w2v = top_100_recommendations_w2v['id'].astype(int).tolist()
mrr_w2v = calculate_mrr(recommended_list_w2v, test_sorted_ids)

print(f"MRR for Word2Vec: {mrr_w2v:.4f}")

MRR for Word2Vec: 0.0217


### 9.3. Tính NDCG (Normalized Discounted Cumulative Gain)

In [29]:
def calculate_dcg_with_ratings(recommended_list, test_movies_with_ratings, k=100):
    # Tạo mapping từ tmdbId -> rating
    rating_map = dict(zip(test_movies_with_ratings['tmdbId'].astype(int), 
                          test_movies_with_ratings['rating']))
    
    dcg = 0.0
    for i, movie_id in enumerate(recommended_list[:k], start=1):
        # Lấy rating làm relevance score (nếu không có thì = 0)
        relevance = rating_map.get(movie_id, 0.0)
        if relevance > 0:
            # Công thức DCG chuẩn: rel_i / log2(i+1)
            dcg += relevance / np.log2(i + 1)
    
    return dcg

def calculate_ndcg_with_ratings(recommended_list, test_movies_with_ratings, k=100):
    # DCG của recommendation hiện tại
    dcg = calculate_dcg_with_ratings(recommended_list, test_movies_with_ratings, k)
    
    # IDCG: DCG của ranking lý tưởng (sắp xếp theo rating giảm dần)
    ideal_test = test_movies_with_ratings.sort_values('rating', ascending=False)
    ideal_list = ideal_test['tmdbId'].astype(int).tolist()
    idcg = calculate_dcg_with_ratings(ideal_list, test_movies_with_ratings, k)
    
    if idcg == 0:
        return 0.0
    
    return dcg / idcg

## 10. So sánh và Kết luận

In [30]:
# ===== TÍNH NDCG SỬ DỤNG RATINGS =====
print("=" * 70)
print("TÍNH NDCG VỚI RATING-BASED RELEVANCE")

# Tính NDCG cho TF-IDF (sử dụng ratings)
ndcg_tfidf_ratings = calculate_ndcg_with_ratings(recommended_list_tfidf, test_movies, k=100)
print(f"NDCG@100 for TF-IDF (rating-based): {ndcg_tfidf_ratings:.4f}")

# Tính NDCG cho Word2Vec (sử dụng ratings)
ndcg_w2v_ratings = calculate_ndcg_with_ratings(recommended_list_w2v, test_movies, k=100)
print(f"NDCG@100 for Word2Vec (rating-based): {ndcg_w2v_ratings:.4f}")

TÍNH NDCG VỚI RATING-BASED RELEVANCE
NDCG@100 for TF-IDF (rating-based): 0.0000
NDCG@100 for Word2Vec (rating-based): 0.0062


In [32]:
# Tạo bảng so sánh
comparison_df = pd.DataFrame({
    'Metric': ['Precision@100', 'Recall@100', 'F1@100', 'MRR', 'NDCG@100 (Ratings)'],
    'TF-IDF': [
        f"{metrics_tfidf['precision']:.4f}",
        f"{metrics_tfidf['recall']:.4f}",
        f"{metrics_tfidf['f1']:.4f}",
        f"{mrr_tfidf:.4f}",
        f"{ndcg_tfidf_ratings:.4f}"
    ],
    'Word2Vec': [
        f"{metrics_w2v['precision']:.4f}",
        f"{metrics_w2v['recall']:.4f}",
        f"{metrics_w2v['f1']:.4f}",
        f"{mrr_w2v:.4f}",
        f"{ndcg_w2v_ratings:.4f}"
    ]
})

print("BẢNG SO SÁNH KẾT QUẢ: TF-IDF vs WORD2VEC")
display(comparison_df)

BẢNG SO SÁNH KẾT QUẢ: TF-IDF vs WORD2VEC


,Metric,TF-IDF,Word2Vec
0,Precision@100,0.0000,0.0100
1,Recall@100,0.0000,0.0030
2,F1@100,0.0000,0.0046
3,MRR,0.0000,0.0217
4,NDCG@100 (Ratings),0.0000,0.0062


## 11. Nhận xét và Kết luận

### Nhận xét:

#### 1. **So sánh hiệu suất:**

**TF-IDF:**
- **Ưu điểm:**
  - Nhanh và hiệu quả về mặt tính toán
  - Dễ hiểu và triển khai
  - Hiệu quả với dữ liệu có cấu trúc rõ ràng
  - Bắt được các từ khóa quan trọng và đặc trưng

- **Nhược điểm:**
  - Không bắt được ý nghĩa ngữ nghĩa (semantic meaning)
  - Không xử lý được từ đồng nghĩa
  - Ma trận thưa (sparse matrix) có thể dẫn đến mất thông tin

**Word2Vec:**
- **Ưu điểm:**
  - Bắt được mối quan hệ ngữ nghĩa giữa các từ
  - Xử lý tốt từ đồng nghĩa và tương tự
  - Vector dense (không thưa) giữ được nhiều thông tin
  - Có khả năng generalize tốt hơn

- **Nhược điểm:**
  - Yêu cầu corpus lớn để train tốt
  - Phức tạp và tốn thời gian hơn
  - Có thể không hiệu quả với corpus nhỏ